In [1]:
from dotenv import load_dotenv
import os
import json
from pypdf import PdfReader
import gradio as gr
import smtplib
from google import genai
from google.genai.types import Tool, FunctionDeclaration, Schema, GenerateContentConfig


c:\Users\rumes\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)

True

In [3]:
def push(message, reason):
    print(f"Push: {message}")
    
    # Load environment variables inside the function
    push_mail = os.getenv("CLONE_EMAIL")
    push_password = os.getenv("CLONE_PASSWORD")
    
    print(f"Email: {push_mail}")
    print(f"Password: {'*' * len(push_password) if push_password else 'None'}")
    
    if not push_mail or not push_password:
        print("Error: CLONE_EMAIL or CLONE_PASSWORD not found in environment variables")
        print("Please create a .env file with CLONE_EMAIL and CLONE_PASSWORD")
        return
    
    try:
        mail=smtplib.SMTP("smtp.gmail.com")
        mail.starttls()
        mail.login(user=push_mail, password=push_password)
        mail.sendmail(
            from_addr=push_mail, 
            to_addrs="rumesefia@gmail.com", 
            msg=f"Subject:{reason}\n\n{message}"
            )
        mail.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")


In [4]:
def record_user_details(email, name="Name not provided", notes="notes not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}", "Recording User Details")
    return {"recorded": "ok"}

In [5]:
def record_unknown_question(question):
    push(f"Recording {question} that I don't know the answer to", "Recording Unknown Question")
    return {"recorded": "ok"}

In [6]:
tools_list = [
    {
      "function_declarations": [
        {
          "name": "record_user_details",
          "description": "Records user details including email, name, and notes. Used for capturing user interest or contact information.",
          "parameters": {
            "type": "OBJECT",
            "properties": {
              "email": {
                "type": "STRING",
                "description": "The user's email address (required)."
              },
              "name": {
                "type": "STRING",
                "description": "The user's name (optional, defaults to 'Name not provided')."
              },
              "notes": {
                "type": "STRING",
                "description": "Any additional notes or details provided by the user (optional, defaults to 'notes not provided')."
              }
            },
            "required": ["email"]
          }
        }
      ]
    },
    {
      "function_declarations": [
        {
          "name": "record_unknown_question",
          "description": "Records a question that the Gemini agent does not know the answer to, for future reference or training.",
          "parameters": {
            "type": "OBJECT",
            "properties": {
              "question": {
                "type": "STRING",
                "description": "The question that the agent could not answer."
              }
            },
            "required": ["question"]
          }
        }
      ]
    }
]


In [7]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [8]:
reader = PdfReader("Profile.pdf")

linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text


In [9]:
resume_reader = PdfReader("resume.pdf")

resume = ""
for page in resume_reader.pages:
    text = page.extract_text()
    if text:
        resume += text

with open("summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

name = "Sefia Oghenerume"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background, current professional resume, and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool."

system_prompt += f"Respond in a natural, conversational, and engaging tone. Avoid using bullet points, markdown formatting, or lists, \
    unless explicitly requested or required to drive home a point. Write as if you are having a friendly, semi-formal \
    chat with the user—use contractions, vary your sentence structure, and keep your language warm and personable."

system_prompt += f"You are to represent the engineer of this agent and the author of this prompt {name} \
    prioritize the information in the summary although not openly divulge it to the user. \
    it is simply a guide to give context of the person you are deployed to represent. \
    Following the summary, prioritise information in the resume, followed by the github stated in the summary, then the linkedin last"

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n## Resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [11]:
client  =  genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))


In [12]:
response = client.models.generate_content(model="gemini-2.0-flash", contents="What is the capital of France?")
print(response.text)

The capital of France is Paris.



In [13]:
from google.genai.types import GenerateContentConfig

def chat(message, history):
    config = GenerateContentConfig(tools=tools_list)
    gemini_messages = []
    if system_prompt:
        gemini_messages.append({"role": "user", "parts": [{"text": system_prompt}]})
    for h_msg in history:
        if h_msg["role"] == "user":
            gemini_messages.append({"role": "user", "parts": [{"text": h_msg["content"]}]})
        elif h_msg["role"] == "assistant":
            parts = []
            if "content" in h_msg and h_msg["content"]:
                parts.append({"text": h_msg["content"]})
            if "tool_calls" in h_msg and h_msg["tool_calls"]:
                for tc in h_msg["tool_calls"]:
                    parts.append({"function_call": {"name": tc.function.name, "args": json.loads(tc.function.arguments)}})
            gemini_messages.append({"role": "model", "parts": parts})
        elif h_msg["role"] == "tool":
            gemini_messages.append(h_msg)
    gemini_messages.append({"role": "user", "parts": [{"text": message}]})

    done = False
    while not done:
        response = client.models.generate_content(model="gemini-2.0-flash", contents=gemini_messages, config=config)
        
        tool_calls_from_gemini = []
        candidates = getattr(response, "candidates", None)
        if candidates and candidates[0].content:
            parts = getattr(candidates[0].content, "parts", None)
        tool_calls_from_gemini = [
            part.function_call for part in parts if getattr(part, "function_call", None)
        ]

        if tool_calls_from_gemini and response.candidates[0].content.parts:
            gemini_messages.append({"role": "model", "parts": response.candidates[0].content.parts})
            results = handle_tool_calls(tool_calls_from_gemini)
            gemini_messages.extend(results)
        else:
            done = True

    final_text_parts = [part.text for part in response.candidates[0].content.parts if part.text]
    return final_text_parts[0] if final_text_parts else "" 



In [14]:
gr.ChatInterface(chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [15]:
# def handle_tool_calls(tool_calls):
#     results = []
#     for tool_call in tool_calls:
#         tool_name = tool_call.name
#         arguments = tool_call.args
#         print(f"Tool called: {tool_name} with arguments: {arguments}", flush=True)

#         tool = globals().get(tool_name)
#         result_data = tool(**arguments) if tool else {"error": f"Tool '{tool_name}' not found."}

#         results.append({
#             "role": "function",
#             "parts": [{"function_response": {"name": tool_name, "response": result_data}}]
#         })
#     return results

    #GOOD ONE
    #But can be better

In [16]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
#     done = False
#     while not done:
#         response = client.models.generate_content(model="gemini-2.0-flash", contents=messages, config=config)
#         finish_reason = response.choices[0].finish_reason
#         if finish_reason=="tool_calls":
#             message = response.choices[0].message
#             tool_calls = message.tool_calls
#             results = handle_tool_calls(tool_calls)
#             messages.append(message)
#             messages.extend(results)
#         else:
#             done = True
#     return messages[-1]["content"]

In [17]:
# def chat(message, history):
#     gemini_messages = []
#     if system_prompt:
#         gemini_messages.append({"role": "user", "parts": [{"text": system_prompt}]})

#     for h_msg in history:
#         if h_msg["role"] == "user":
#             gemini_messages.append({"role": "user", "parts": [{"text": h_msg["content"]}]})
#         elif h_msg["role"] == "assistant":
#             parts = []
#             if "content" in h_msg and h_msg["content"]:
#                 parts.append({"text": h_msg["content"]})
#             if "tool_calls" in h_msg and h_msg["tool_calls"]:
#                 for tc in h_msg["tool_calls"]:
#                     parts.append({"function_call": {"name": tc.function.name, "args": json.loads(tc.function.arguments)}})
#             gemini_messages.append({"role": "model", "parts": parts})
#         elif h_msg["role"] == "tool":
#             gemini_messages.append(h_msg)

#     gemini_messages.append({"role": "user", "parts": [{"text": message}]})

#     final_response_content = ""
#     done = False

#     while not done:
#         try:
#             response = client.models.generate_content(model="gemini-2.0-flash", contents=gemini_messages)

#             if not response.candidates or not response.candidates[0].content:
#                 print("ERROR: Gemini response is empty or malformed.")
#                 final_response_content = "I encountered an issue getting a response from the AI."
#                 done = True
#                 break

#             model_response_parts = response.candidates[0].content.parts
#             if model_response_parts:
#                 tool_calls_from_gemini = [
#                     part.function_call for part in model_response_parts if hasattr(part, "function_call")
#                 ]

#                 if tool_calls_from_gemini:
#                     gemini_messages.append({"role": "model", "parts": model_response_parts})
#                     tool_execution_results = handle_tool_calls(tool_calls_from_gemini)
#                     gemini_messages.extend(tool_execution_results)
#                 else:
#                     final_response_content = "".join([
#                         part.text for part in model_response_parts if hasattr(part, "text") and part.text is not None
#                     ])
#                     done = True

#         except Exception as e:
#             print(f"ERROR: An unexpected error occurred during chat: {e}")
#             final_response_content = f"I encountered an unexpected error: {e}"
#             done = True

#     return final_response_content


In [ ]:

# # 1. Prepare tools and config
# tools = [
#     types.Tool(function_declarations=[
#         record_unknown_question_json,
#         record_user_details_json
#     ])
# ]
# config = types.GenerateContentConfig(tools=tools)

# # 2. The chat function
# def chat(message, history):
#     # Compose the conversation
#     contents = [
#         types.Content(role="user", parts=[types.Part(text=message)])
#     ]
#     # Add history if needed (optional, for multi-turn)
#     # for user_msg, bot_msg in history:
#     #     contents.append(types.Content(role="user", parts=[types.Part(text=user_msg)]))
#     #     contents.append(types.Content(role="model", parts=[types.Part(text=bot_msg)]))

#     # 3. Call the model
#     response = genai.Client().models.generate_content(
#         model="gemini-2.5-flash",  # or your preferred model
#         contents=contents,
#         config=config,
#     # )

    # 4. Check for function call
    # part = response.candidates[0].content.parts[0]
    # if hasattr(part, "function_call") and part.function_call:
    #     fn = part.function_call
    #     # Map function name to your Python function
    #     fn_map = {
    #         "record_unknown_question": record_unknown_question,
    #         "record_user_details": user_details,
    #     }
    #     result = fn_map[fn.name](**fn.args)
    #     # 5. Send function result back to model for final response
    #     function_response_part = types.Part.from_function_response(
    #         name=fn.name,
    #         response={"result": result},
    #     )
    #     contents.append(response.candidates[0].content)  # model's function call message
    #     contents.append(types.Content(role="user", parts=[function_response_part]))
    #     final_response = genai.Client().models.generate_content(
    #         model="gemini-2.5-flash",
    #         contents=contents,
    #         config=config,
    #     )
    #     return final_response.text
    # else:
    #     # No function call, just return the text
    #     return response.text

In [19]:
# def chat(message, history):
#     # Gemini needs the full conversation as a single string
#     conversation = f"System: {system_prompt}\n"
#     for user_msg, bot_msg in history:
#         conversation += f"User: {user_msg}\nAssistant: {bot_msg}\n"
#     conversation += f"User: {message}\nAssistant:"

#     model = genai.GenerativeModel("gemini-1.5-flash-latest", tools=tools)

#     # Generate response with tool support
#     response = model.generate_content([conversation], tool_config={"enable_tool_calling": True})

#     # If Gemini wants to call a tool, handle it
#     if hasattr(response, "candidates") and response.candidates:
#         candidate = response.candidates[0]
#         if hasattr(candidate, "content") and candidate.content:
#             # Check for tool calls in the response
#             tool_calls = getattr(candidate, "tool_calls", None)
#             if tool_calls:
#                 # Run the tool(s) and append results to conversation
#                 results = handle_tool_calls(tool_calls)
#                 # Optionally, you can append tool results to the conversation and re-query Gemini for a final answer
#                 # For simplicity, return the tool result or a message
#                 return results[0]["content"] if results else "Tool called, but no result."
#             else:
#                 return candidate.content.parts[0].text if candidate.content.parts else candidate.content.text
#     # Fallback: return the plain response text
#     return response.text